<a href="https://colab.research.google.com/github/rahiakela/automl-experiments/blob/main/auto-keras-practice-works/02_image_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Image Regression

To make this tutorial easy to follow, we just treat MNIST dataset as a regression dataset. It means we will treat prediction targets of MNIST dataset, which are integers ranging from 0 to 9 as numerical values, so that they can be directly used as the regression targets.

Reference:

https://autokeras.com/tutorial/image_regression/

In [1]:
!pip -q install autokeras

     |████████████████████████████████| 166 kB 16.3 MB/s 
     |████████████████████████████████| 454.4 MB 21 kB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 4.0 MB 22.5 MB/s 
     |████████████████████████████████| 462 kB 47.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.4 MB/s 


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

##A Simple Example

The first step is to prepare your data. Here we use the MNIST dataset as an example.

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[:100]
y_train = y_train[:100]
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])    # array([7, 2, 1], dtype=uint8)

11493376/11490434 [==============================] - 0s 0us/step
(100, 28, 28)
(100,)
[5 0 4]


The second step is to run the ImageRegressor. It is recommended have more trials for more complicated datasets. This is just a quick demo of MNIST, so we set max_trials to 1. 

For the same reason, we set epochs to 2. You can also leave the epochs unspecified for an adaptive number of epochs.

In [ ]:
# Initialize the image regressor
reg = ak.ImageRegressor(overwrite=True, max_trials=1)
# Feed the image regressor with training data
reg.fit(x_train, y_train, epochs=2)

In [7]:
# Predict with the best model
predicted_y = reg.predict(x_test)
print(predicted_y)

313/313 [==============================] - 43s 138ms/step
[[0.14437975]
 [0.62272555]
 [0.15234606]
 ...
 [0.15122914]
 [0.171411  ]
 [0.23554677]]


In [8]:
# Evaluate the best model with testing data
print(reg.evaluate(x_test, y_test))

313/313 [==============================] - 44s 137ms/step - loss: 26.5229 - mean_squared_error: 26.5229
[26.52287483215332, 26.52287483215332]


##Validation Data

By default, AutoKeras use the last 20% of training data as validation data. 

As shown in the example below, you can use `validation_split` to specify the percentage.

In [ ]:
reg.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=2,
)

You can also use your own validation set instead of splitting it from the training data with validation_data.

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]

reg.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=2,
)

##Customized Search Space

For advanced users, you may customize your search space by using AutoModel instead of ImageRegressor. You can configure the ImageBlock for some high-level configurations, e.g., block_type for the type of neural network to search, normalize for whether to do data normalization, augment for whether to do data augmentation. You can also do not specify these arguments, which would leave the different choices to be tuned automatically.

See the following example for detail.

In [9]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    block_type="resnet",  # Only search ResNet architectures
    normalize=False,      # Normalize the dataset
    augment=False         # Do not do data augmentation
)(input_node)

output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 53s]
val_loss: 15.975235939025879

Best val_loss So Far: 15.975235939025879
Total elapsed time: 00h 00m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
4/4 [==============================] - 26s 3s/step - loss: 42.1530 - mean_squared_error: 42.1530
Epoch 2/2
4/4 [==============================] - 15s 3s/step - loss: 51.0456 - mean_squared_error: 51.0456


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


The usage of AutoModel is similar to the functional API of Keras. Basically, you are building a graph, whose edges are blocks and the nodes are intermediate outputs of blocks. To add an edge from input_node to `output_node` with `output_node = ak.some_block(input_node)`.

You can even also use more fine grained blocks to customize the search space even further. See the following example.

In [10]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 27s]
val_loss: 27.605497360229492

Best val_loss So Far: 27.605497360229492
Total elapsed time: 00h 00m 27s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
4/4 [==============================] - 13s 2s/step - loss: 21.2293 - mean_squared_error: 21.2293
Epoch 2/2
4/4 [==============================] - 8s 2s/step - loss: 8.8052 - mean_squared_error: 8.8052


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


##Data Format

The AutoKeras ImageRegressor is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel dimension. The images in the MNIST dataset do not have the channel dimension. Each image is a matrix with shape (28, 28). AutoKeras also accepts images of three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28, 28, 1).

For the regression targets, it should be a vector of numerical values. AutoKeras accepts numpy.ndarray.

We also support using tf.data.Dataset format for the training data. In this case, the images would have to be 3-dimentional.

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
y_train = y_train.reshape(y_train.shape + (1,))
y_test = y_test.reshape(y_test.shape + (1,))

print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)

(60000, 28, 28, 1)
(60000, 1)


In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

reg = ak.ImageRegressor(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=2)
# Predict with the best model.
predicted_y = reg.predict(test_set)
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))